# Jobsheet 15 - Pipeline ML & Deployment

Nama: Afrizal Qurratul Faizin

NIM: 2341720083
## Praktikum 2


In [1]:
import os
import numpy as np
import cv2
import pickle
import tensorflow as tf
from flask import Flask, request, render_template_string
from skimage.feature import hog

app = Flask(__name__)

# Load Model & Scaler
MODEL_PATH = 'day_night_model.h5'
SCALER_PATH = 'scaler.pkl'

try:
    model = tf.keras.models.load_model(MODEL_PATH)
    with open(SCALER_PATH, 'rb') as f:
        scaler = pickle.load(f)
    print("✅ System Loaded Successfully")
except Exception as e:
    print(f"❌ Error loading system: {e}")

def preprocess_image(image_bytes):
    # Decode gambar
    nparr = np.frombuffer(image_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    # Preprocessing (Harus sama persis dengan Training)
    img = cv2.resize(img, (256, 256))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    hog_feat = hog(gray, orientations=9, pixels_per_cell=(8,8),
                   cells_per_block=(2,2), block_norm='L2-Hys',
                   visualize=False, feature_vector=True)
                   
    return scaler.transform(hog_feat.reshape(1, -1))

@app.route('/', methods=['GET'])
def home():
    return render_template_string('''
    <div style="text-align:center; padding:50px;">
        <h1>Day vs Night Classifier</h1>
        <form action="/predict" method="post" enctype="multipart/form-data">
            <input type="file" name="file" required><br><br>
            <button type="submit">Prediksi</button>
        </form>
    </div>
    ''')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        file = request.files['file']
        data = preprocess_image(file.read())
        prediction = model.predict(data)[0][0]
        
        label = "Day (Siang)" if prediction > 0.5 else "Night (Malam)"
        return f"<h2 style='text-align:center'>Hasil: {label}</h2><center><a href='/'>Kembali</a></center>"
    except Exception as e:
        return f"Error: {e}"

if __name__ == '__main__':
    # Port 7860 wajib untuk Hugging Face Spaces
    app.run(host='0.0.0.0', port=7860)

✅ System Loaded Successfully
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:7860
 * Running on http://192.168.1.44:7860
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:57:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:57:58] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 607ms/step


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:58:11] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:58:13] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step


...

INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:58:41] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:58:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:58:51] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:58:55] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:59:03] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:59:04] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 16:59:17] "POST /predict HTTP/1.1" 200 -


## Hasil Publish:

https://huggingface.co/spaces/afqufaipolygonal/daynight-classifier-afrizalqf